In [2]:
import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sb

from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.models import Word2Vec, Phrases

In [4]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import accuracy_score, confusion_matrix

import os

Using TensorFlow backend.


In [20]:
%ls *.tsv

 Volume in drive C is OS
 Volume Serial Number is EC4E-A0E4

 Directory of C:\Users\xiangyangcao\Documents

06/27/2019  01:52 PM        33,556,378 labeledTrainData.tsv
06/13/2019  04:30 PM         3,367,149 test.tsv
06/27/2019  01:52 PM        32,724,746 testData.tsv
06/13/2019  04:30 PM         8,481,022 train.tsv
06/27/2019  01:52 PM        67,281,491 unlabeledTrainData.tsv
               5 File(s)    145,410,786 bytes
               0 Dir(s)  136,888,295,424 bytes free


### Preview on datasets (wordy...boring )

* labeledTrainData
```
$ head labeledTrainData.tsv -n 2
id      sentiment       review
"5814_8"        1       "With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched T..."
```

* unlabeledTrainData
```
$ head unlabeledTrainData.tsv -n 2
id      review
"9999_0"        "Watching Time Chasers, it obvious that it was made by a bunch of friends. Maybe they were sitting around one day in film school and said, \"Hey, let's pool our money together and make a really bad movie!\" Or something like that. What ever they said, they still ended up making a really bad movie--dull story, bad script, lame acting, poor cinematography, bottom of the barrel stock music, etc. All corners were cut, except the one that would have prevented this film's release. Life's like that."
```

* imdb_master.csv
```
$ head imdb_master.csv -n 2
,type,review,label,file
0,test,"Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.",neg,0_2.txt
```

In [41]:
usecols = ['sentiment','review']


In [42]:
usecols = ['sentiment','review']

train_data = pd.read_csv(
    filepath_or_buffer='./imdb/labeledTrainData.tsv',
    usecols=usecols, sep='\t')


In [43]:
additional_data = pd.read_csv(filepath_or_buffer='./imdb/imdb_master.csv', encoding = "latin-1")

In [44]:
unlabeled_data = pd.read_csv(
    filepath_or_buffer="./imdb/unlabeledTrainData.tsv", 
    error_bad_lines=False,
    sep='\t')

b'Skipping line 43043: expected 2 fields, saw 3\n'


In [45]:
submission_data = pd.read_csv(
    filepath_or_buffer="./imdb/testData.tsv",
    sep='\t')

In [46]:
train_data.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


In [47]:
additional_data.head()

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt
